In [1]:
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from sklearn.metrics import f1_score
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
from sklearn.model_selection import train_test_split



pd.options.mode.chained_assignment = None

In [2]:
# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

# import psutil
# import humanize
# import os
# import GPUtil as GPU

# GPUs = GPU.getGPUs()
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), 
#           " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(
#         gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [3]:
torch.cuda.empty_cache()
#printm()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#!ls '/content/drive/My Drive/app'

In [5]:
mimic_data = pd.read_csv("/content/drive/My Drive/app/text_binary.csv")
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,59835582,16043746,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,51487790,16456872,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,59750073,16824069,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
mimic_data = mimic_data.loc[mimic_data['No Finding'] == 0]
mimic_data.head()

,study_id,subject_id,text,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,58522792,16567081,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,58213163,16567081,b' FINAL REPOR...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,50629293,16059088,"b"" FINAL REPOR...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
6,50113713,16059088,b' WET READ: ___ ___ ___ 7:27 PM\n Left PICC ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
7,57620089,16059088,b' FINAL REPOR...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
binary_data = mimic_data[['text','Pleural Effusion']]
binary_data.text = binary_data.text.str.lower()
binary_data.head()

,text,Pleural Effusion
0,"b"" final repor...",0.0
1,b' final repor...,1.0
5,"b"" final repor...",0.0
6,b' wet read: ___ ___ ___ 7:27 pm\n left picc ...,0.0
7,b' final repor...,0.0


In [8]:
binary_data.groupby('Pleural Effusion').count()

,text
Pleural Effusion,
0.0,98072
1.0,54300


In [9]:
negative = binary_data.loc[binary_data['Pleural Effusion']==0]
print(len(negative))
positive = binary_data.loc[binary_data['Pleural Effusion']==1].sample(n=len(negative),replace=True)

98072


In [10]:
binary_data = positive.append(negative,ignore_index=True).reset_index(drop=True)
#df1.append(df2, ignore_index=True)

In [11]:
len(binary_data)

196144

In [12]:
binary_data.text[0]

"b'                                 final report\\n pa and lateral chest, ___ at 20:22\\n \\n clinical indication:  ___-year-old postop day #2 from robotic vats wedge\\n resection, decreased saturations in high 80s with ambulation.  evaluate for\\n interval change.\\n \\n comparison to ___ at 10:36.\\n \\n pa and lateral views of the chest, ___ at 20:22 is submitted.\\n \\n impression:\\n 1.  small right apical pneumothorax is unchanged with post-surgical changes at\\n the apex and small opacity in the apex medially, which has slightly improved,\\n suggesting improving aeration and postoperative changes.  there is blunting of\\n the right costophrenic angle, likely reflecting a small effusion.  left lung\\n remains grossly clear.  rounded nodular opacity at the left base likely\\n reflects a nipple shadow.  left basilar linear opacity likely represents\\n subsegmental atelectasis.  overall, the right lateral chest wall subcutaneous\\n emphysema has resolved.  overall, cardiac and media

In [13]:
y = binary_data['Pleural Effusion']
X = binary_data.drop(columns=['Pleural Effusion'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [14]:
X_train.head()

,text
0,b' final repor...
1,b' final repor...
2,b' final repor...
3,b' final repor...
4,b' final repor...


In [15]:
# remove all '\\n' from the text
re_newlines = re.compile('\\\\n')
def sub_newlines(x): return re_newlines.sub('',x)

# remove all special characters from the text, keep only alphanumeric and spaces
re_letters = re.compile('[^A-Za-z0-9 ]')
def sub_letters(x): return re_letters.sub('', x)

# remove excessive spacing otherwise you end up with " " substrings
re_spaces = re.compile('\s+')
def sub_spaces(x): return re_spaces.sub(' ', x)
                
# tokenize all words.
my_tok = spacy.load('en')
def spacy_tok(x): 
    return [tok.text for tok in my_tok.tokenizer(sub_spaces
                                                 (sub_letters
                                                 (sub_newlines(x))))]

In [16]:
sub_spaces(sub_letters(sub_newlines(binary_data.text[0])))

'b final report pa and lateral chest at 2022 clinical indication yearold postop day 2 from robotic vats wedge resection decreased saturations in high 80s with ambulation evaluate for interval change comparison to at 1036 pa and lateral views of the chest at 2022 is submitted impression 1 small right apical pneumothorax is unchanged with postsurgical changes at the apex and small opacity in the apex medially which has slightly improved suggesting improving aeration and postoperative changes there is blunting of the right costophrenic angle likely reflecting a small effusion left lung remains grossly clear rounded nodular opacity at the left base likely reflects a nipple shadow left basilar linear opacity likely represents subsegmental atelectasis overall the right lateral chest wall subcutaneous emphysema has resolved overall cardiac and mediastinal contours are stable'

In [17]:
#spacy_tok(binary_data.text[0])[1:]

In [18]:
binary_train = binary_data.copy()
binary_train.head()

,text,Pleural Effusion
0,b' final repor...,1.0
1,b' wet read: ___ ___ 8:00 am\n \n \n \n th...,1.0
2,b' final repor...,1.0
3,b' final repor...,1.0
4,b' wet read: ___ ___ ___ 12:22 am\n no acute ...,1.0


In [19]:
def get_counts(text):
    counts = Counter()
    for word in text:
        counts.update(spacy_tok(word)[1:])
    return counts

In [20]:
word_count = get_counts(binary_train.text)

In [21]:
len(word_count.keys())

31110

In [22]:
for word in list(word_count):
    if word_count[word] < 3:
        del word_count[word]

In [23]:
len(word_count.keys())

16028

In [24]:
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [25]:
len(words)

16030

In [26]:
def encode_sentence(text):
    enc = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in text.split()])
    return enc

In [27]:
encode_sentence(binary_train.text[0])

array([ 1,  2,  1,  4,  5,  6,  1,  1,  8,  1,  1, 10,  1,  1, 13, 14,  1,
       16, 17, 18,  1,  1, 21, 22, 23, 24, 25, 26,  1, 28,  1, 30,  1,  1,
       32, 33,  1,  8,  1,  1,  4,  5,  6, 35, 36, 37,  1,  1,  8,  1, 38,
        1,  1,  1,  1, 42, 43, 44, 45, 38, 46, 26,  1, 48,  1, 37, 49,  5,
       42, 50, 23, 37, 49,  1, 52, 53, 54,  1, 56, 57, 58,  5, 59,  1, 60,
       38, 61,  1, 37, 43, 62,  1, 64, 65, 66, 42,  1, 68,  1, 70, 71,  1,
       73, 74, 50,  8, 37, 68, 75,  1, 76, 66, 77,  1, 68, 79, 80, 50, 64,
        1, 82,  1,  1, 37, 43,  6,  7, 85,  1, 87, 53,  1,  1, 89,  5, 90,
       91, 92,  1])

## Dataset and data loaders

In [28]:
class Binary_Mimic(Dataset):
    def __init__(self, X, y, vocab):
        self.x = [encode_sentence(x) for x in X.text]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        return x, y

In [29]:
b_train = Binary_Mimic(X_train, y_train, vocab2index)
b_valid = Binary_Mimic(X_val, y_val, vocab2index)

In [30]:
def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (sentences, labels).
    
    Need custom collate_fn because merging sequences (including padding) is not 
    supported in default. Sequences are padded to the maximum length of mini-batch 
    sequences (dynamic padding).
    
    Args:
        data: list of tuple (sentence, label). 
            - list of word indices of variable length
            - label, 0 or 1
    Returns:
        packed_batch: (PackedSequence), see torch.nn.utils.rnn.pack_padded_sequence
        sencences: torch tensor of shape (batch_size, max_len).
        labels: torch tensor of shape (batch_size, 1).
        lengths: list; valid length for each padded sentence. 
    """
    # Sort a data list by sentences length (descending order).
    data.sort(key=lambda x: len(x[0]), reverse=True)
    sentences, labels = zip(*data)
    
    # stack labels
    labels = torch.Tensor(labels)
    
    # Merge sentences
    lengths = [len(s) for s in sentences]
   
    sents = torch.zeros(len(sentences), max(lengths)).long()
    for i, s in enumerate(sentences):
        end = lengths[i]
        sents[i, :end] = torch.Tensor(s[:end])        
    
    return sents, lengths, labels

In [31]:
# data = [b_train[0], b_train[1], b_train[2]]
# data

In [32]:
# collate_fn(data)

In [33]:
# train_loader = DataLoader(b_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
# sents, lengths, labels = next(iter(train_loader))
# sents

In [34]:
# sents.shape, labels.shape

In [35]:
# lengths

## Training

In [36]:
def train_epocs(model, optimizer, train_dl, valid_dl, epochs=10):
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.cuda().long()
            y = y.cuda().float()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        print("Epoch #%.f: train loss %.3f val loss %.3f and val accuracy %.3f" % 
              (i+1,sum_loss/total, val_loss, val_acc))
  

In [37]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    with torch.no_grad():
      for x, s, y in valid_dl:
          x = x.cuda().long()
          y = y.cuda().float().unsqueeze(1)
          y_hat = model(x, s)
          loss = F.binary_cross_entropy_with_logits(y_hat, y)
          y_pred = y_hat > 0
          correct += (y_pred.float() == y).float().sum()
          total += y.shape[0]
          sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

## Basic GRU Model

In [38]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super(GRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.2)
        
    def forward(self, x, lengths):
        #print(x.shape)
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [39]:
torch.cuda.empty_cache()
#printm()

In [40]:
batch_size = 25000
train_dl = DataLoader(b_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
valid_dl = DataLoader(b_valid, batch_size=batch_size, collate_fn=collate_fn)

In [41]:
vocab_size = len(words)
print(vocab_size)
model = GRUModel(vocab_size, 50, 50).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

16030


In [42]:
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.695 val loss 0.668 and val accuracy 0.595
Epoch #2: train loss 0.644 val loss 0.550 and val accuracy 0.734
Epoch #3: train loss 0.463 val loss 0.364 and val accuracy 0.869
Epoch #4: train loss 0.349 val loss 0.299 and val accuracy 0.896
Epoch #5: train loss 0.293 val loss 0.265 and val accuracy 0.908
Epoch #6: train loss 0.256 val loss 0.237 and val accuracy 0.917
Epoch #7: train loss 0.231 val loss 0.211 and val accuracy 0.923
Epoch #8: train loss 0.208 val loss 0.191 and val accuracy 0.931
Epoch #9: train loss 0.189 val loss 0.171 and val accuracy 0.940
Epoch #10: train loss 0.172 val loss 0.158 and val accuracy 0.943
Epoch #11: train loss 0.157 val loss 0.146 and val accuracy 0.949
Epoch #12: train loss 0.146 val loss 0.137 and val accuracy 0.953
Epoch #13: train loss 0.136 val loss 0.133 and val accuracy 0.955
Epoch #14: train loss 0.129 val loss 0.125 and val accuracy 0.959
Epoch #15: train loss 0.122 val loss 0.120 and val accuracy 0.961


## Use Glove to create pre-trained embedding 

In [43]:
def loadGloveModel(gloveFile='/content/drive/My Drive/app/glove.6B.50d.txt'):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs

In [44]:
word_vecs = loadGloveModel()

In [45]:
word_count = get_counts(binary_train.text)

In [46]:
print(len(word_vecs.keys()), len(word_count.keys()))

400000 31110


In [47]:
def delete_rare_words(word_vecs, word_count, min_df=4):
    """ Deletes rare words from word_count
    
    Deletes words from word_count if they are not in word_vecs
    and don't have at least min_df occurrencies in word_count.
    """
    words_delete = []
    for word in word_count:
        if word_count[word] < min_df and word not in word_vecs:
            words_delete.append(word)
    for word in words_delete: word_count.pop(word)
    return word_count

In [48]:
word_count = delete_rare_words(word_vecs, word_count, min_df=3)
print(len(word_count.keys()))

19626


In [49]:
def create_embedding_matrix(word_vecs, word_count, min_df=4, emb_size=50):
    """Creates embedding matrix from word vectors. """
    word_count = delete_rare_words(word_vecs, word_count, min_df)
    V = len(word_count.keys()) + 2
    vocab2index = {}
    W = np.zeros((V, emb_size), dtype="float32")
    vocab = ["", "UNK"]
    # adding a vector for padding
    W[0] = np.zeros(emb_size, dtype='float32')
    # adding a vector for rare words 
    W[1] = np.random.uniform(-0.25, 0.25, emb_size)
    vocab2index["UNK"] = 1
    i = 2
    for word in word_count:
        if word in word_vecs:
            W[i] = word_vecs[word]
            vocab2index[word] = i
            vocab.append(word)
            i += 1
        else:
            W[i] = np.random.uniform(-0.25,0.25, emb_size)
            vocab2index[word] = i
            vocab.append(word)
            i += 1   
    return W, np.array(vocab), vocab2index

In [50]:
pretrained_weight, vocab, vocab2index = create_embedding_matrix(word_vecs, word_count, min_df=3)

In [51]:
len(pretrained_weight)

19628

In [52]:
emb_size = 50
V = len(pretrained_weight)
emb = nn.Embedding(V, emb_size)
emb.weight.data.copy_(torch.from_numpy(pretrained_weight))

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 2.4863e-01,  8.5700e-02, -2.4631e-01,  ..., -3.0106e-04,
         -9.4504e-02, -2.0691e-01],
        [-4.1236e-01,  6.4930e-01, -5.5848e-01,  ...,  2.6206e-01,
          1.0448e-01, -4.4302e-01],
        ...,
        [-6.1073e-01, -9.4747e-02,  2.7271e-01,  ...,  4.2417e-01,
          9.3173e-01,  2.8073e-01],
        [-1.3213e-01, -3.7493e-01, -5.5167e-01,  ...,  9.3287e-01,
          3.3618e-01, -2.0084e-01],
        [ 8.5468e-01,  1.9161e-01,  9.6274e-02,  ...,  1.7068e-01,
         -2.6158e-01,  5.3498e-01]])

# GRU Model with Pretrained embedding layer:

In [53]:
class GRUModel_pre(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights=None) :
        super(GRUModel_pre,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x, lengths):
        x = self.embeddings(x)
        x = self.dropout(x)
        pack = pack_padded_sequence(x, lengths, batch_first=True)
        out_pack, ht = self.gru(pack)
        return self.linear(ht[-1])

In [54]:
torch.cuda.empty_cache()
#printm()

In [55]:
vocab_size = len(pretrained_weight)
model = GRUModel_pre(vocab_size, 50, 50, glove_weights=pretrained_weight).cuda()

parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)

In [56]:
train_epocs(model, optimizer,train_dl, valid_dl, epochs=5)

Epoch #1: train loss 0.681 val loss 0.616 and val accuracy 0.658
Epoch #2: train loss 0.590 val loss 0.453 and val accuracy 0.827
Epoch #3: train loss 0.482 val loss 0.374 and val accuracy 0.861
Epoch #4: train loss 0.411 val loss 0.328 and val accuracy 0.882
Epoch #5: train loss 0.361 val loss 0.308 and val accuracy 0.887


In [57]:
model.embeddings.weight.requires_grad = True

In [58]:
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.01)
train_epocs(model, optimizer, train_dl, valid_dl, epochs=15)

Epoch #1: train loss 0.579 val loss 0.378 and val accuracy 0.872
Epoch #2: train loss 0.405 val loss 0.312 and val accuracy 0.891
Epoch #3: train loss 0.321 val loss 0.298 and val accuracy 0.898
Epoch #4: train loss 0.295 val loss 0.273 and val accuracy 0.906
Epoch #5: train loss 0.266 val loss 0.244 and val accuracy 0.914
Epoch #6: train loss 0.246 val loss 0.223 and val accuracy 0.922
Epoch #7: train loss 0.225 val loss 0.207 and val accuracy 0.927
Epoch #8: train loss 0.209 val loss 0.188 and val accuracy 0.933
Epoch #9: train loss 0.192 val loss 0.173 and val accuracy 0.939
Epoch #10: train loss 0.178 val loss 0.162 and val accuracy 0.943
Epoch #11: train loss 0.166 val loss 0.150 and val accuracy 0.947
Epoch #12: train loss 0.156 val loss 0.142 and val accuracy 0.950
Epoch #13: train loss 0.145 val loss 0.134 and val accuracy 0.953
Epoch #14: train loss 0.137 val loss 0.128 and val accuracy 0.956
Epoch #15: train loss 0.130 val loss 0.123 and val accuracy 0.959
